In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import normaltest
from statsmodels.tsa.stattools import acf,pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf

In [2]:
df1 = pd.read_csv("./data/SPX_Real.csv", index_col=False)
df1.columns = ['Timestamp', 'Close']
df1.set_index("Timestamp", inplace=True)
df1.index = pd.to_datetime(df1.index, format='%d-%m-%Y')
df2 = pd.read_csv("./data/AAPL_Real.csv")
df2.columns = ['Timestamp', 'Close']
df2.set_index("Timestamp", inplace=True)
df2.index = pd.to_datetime(df2.index, format='%d-%m-%Y')
df3 = pd.read_csv("./data/TWSE_Real.csv")
df3.columns = ['Timestamp', 'Close']
df3.set_index("Timestamp", inplace=True)
df3.index = pd.to_datetime(df3.index, format='%m/%d/%Y')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def create_dataset(series, window_size, pred_length):
    X, y = [], []
    for i in range(len(series) - window_size - pred_length + 1):
        X.append(series[i : i + window_size])
        y.append(series[i + window_size : i + window_size + pred_length])
    

    return np.array(X), np.array(y)

def get_data_for_period(df, window_size = 20, pred_length = 1):    
    val_start = int(len(df) * 0.8)
    test_start = val_start + int(len(df) * 0.1)
    train_data = df.iloc[:val_start]
    val_data   = df.iloc[val_start:test_start]
    test_data  = df.iloc[test_start:]

    scaler = MinMaxScaler(feature_range=(0, 1))
    train_values = train_data['Close'].values.reshape(-1, 1)
    scaler.fit(train_values)
    
    train_scaled = scaler.transform(train_data['Close'].values.reshape(-1, 1)).flatten()
    val_scaled   = scaler.transform(val_data['Close'].values.reshape(-1, 1)).flatten()
    test_scaled  = scaler.transform(test_data['Close'].values.reshape(-1, 1)).flatten()
    
    X_train, y_train = create_dataset(train_scaled, window_size, pred_length)
    X_val, y_val     = create_dataset(val_scaled, window_size, pred_length)
    X_test, y_test   = create_dataset(test_scaled, window_size, pred_length)
    
    return (X_train, y_train), (X_val, y_val), (X_test, y_test), scaler

In [4]:
dfs = {'SPX': df1, 'AAPL': df2, 'TWSE': df3}

In [ ]:
import torch
from torch.utils.data import Dataset
class TimeSeriesDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X  
        self.Y = Y  

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        past_vals = torch.tensor(self.X[idx], dtype=torch.float32).unsqueeze(-1)
        future_vals = torch.tensor(self.Y[idx], dtype=torch.float32).unsqueeze(-1)

        return {
            "past_values": past_vals,     
            "future_values": future_vals   
        }


In [ ]:
from transformers import PatchTSMixerConfig, PatchTSMixerForPrediction
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader

res = {}
window_size = 55
for name in dfs:
    for pred_length in [1, 5, 10]:
        (X_train, y_train), (X_val, y_val), (X_test, y_test), scaler = get_data_for_period(dfs[name], window_size, pred_length)
        train_dataset = TimeSeriesDataset(X_train, y_train)
        valid_dataset = TimeSeriesDataset(X_val, y_val)
        test_dataset  = TimeSeriesDataset(X_test, y_test)

        loader = DataLoader(train_dataset, batch_size=32)
        batch = next(iter(loader))
        config = PatchTSMixerConfig(context_length = 55, prediction_length = pred_length, num_input_channels=1)
        model = PatchTSMixerForPrediction(config)
        training_args = TrainingArguments(
            output_dir=f"./20_pochs_patch_tsm_results_{name}_pred_length_{pred_length}",
            evaluation_strategy="epoch",
            per_device_train_batch_size=32,
            per_device_eval_batch_size=32,
            num_train_epochs=20,
            auto_find_batch_size = True,
            lr_scheduler_type = "cosine",
            logging_steps=10,
            save_strategy="epoch",
            label_smoothing_factor = 0.1
        )

        trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,   
        )

        trainer.train()

        results = trainer.evaluate(test_dataset)
        print("Test results:", results)
        res[f"{name}_pred_length_{pred_length}"] = results

print(res)


/Users/sunilcotterill/Library/Python/3.10/lib/python/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
[codecarbon ERROR @ 05:48:20] Error: Another instance of codecarbon is already running. Turn off the other instance to be able to run this one. Exiting.


past_values batch shape: torch.Size([32, 55, 1])
future_values batch shape: torch.Size([32, 1, 1])


[codecarbon WARNING @ 05:48:20] Another instance of codecarbon is already running. Exiting.


Epoch,Training Loss,Validation Loss
1,0.000200,0.002938
2,0.000200,0.002344
3,0.000200,0.001816
4,0.000200,0.001429
5,0.000100,0.001186
6,0.000100,0.001056
7,0.000100,0.000978
8,0.000100,0.000925
9,0.000100,0.000884
10,0.000200,0.000853


[codecarbon WARNING @ 05:51:51] Another instance of codecarbon is already running. Exiting.


...

Test results: {'eval_loss': 0.002614461351186037, 'eval_runtime': 0.3926, 'eval_samples_per_second': 2149.601, 'eval_steps_per_second': 68.767, 'epoch': 20.0}
past_values batch shape: torch.Size([32, 55, 1])
future_values batch shape: torch.Size([32, 10, 1])


[codecarbon WARNING @ 06:17:37] Another instance of codecarbon is already running. Exiting.


Epoch,Training Loss,Validation Loss
1,0.002200,0.003341
2,0.001500,0.002813
3,0.001500,0.002419
4,0.001400,0.002180
5,0.001300,0.002034
6,0.001400,0.001941
7,0.001200,0.001874
8,0.001400,0.001818
9,0.001100,0.001775
10,0.001200,0.001740


[codecarbon WARNING @ 06:21:21] Another instance of codecarbon is already running. Exiting.


Test results: {'eval_loss': 0.0037439947482198477, 'eval_runtime': 0.3138, 'eval_samples_per_second': 2673.925, 'eval_steps_per_second': 86.05, 'epoch': 20.0}
{'SPX_pred_length_1': {'eval_loss': 0.0012304793344810605, 'eval_runtime': 0.2709, 'eval_samples_per_second': 3060.148, 'eval_steps_per_second': 95.976, 'epoch': 20.0}, 'SPX_pred_length_5': {'eval_loss': 0.001950511010363698, 'eval_runtime': 0.2958, 'eval_samples_per_second': 2788.615, 'eval_steps_per_second': 87.884, 'epoch': 20.0}, 'SPX_pred_length_10': {'eval_loss': 0.002737656468525529, 'eval_runtime': 0.4681, 'eval_samples_per_second': 1751.657, 'eval_steps_per_second': 55.54, 'epoch': 20.0}, 'AAPL_pred_length_1': {'eval_loss': 0.015614773146808147, 'eval_runtime': 0.2764, 'eval_samples_per_second': 2999.017, 'eval_steps_per_second': 94.058, 'epoch': 20.0}, 'AAPL_pred_length_5': {'eval_loss': 0.02598980814218521, 'eval_runtime': 0.2976, 'eval_samples_per_second': 2771.713, 'eval_steps_per_second': 87.351, 'epoch': 20.0}, 'AA

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from transformers import PatchTSMixerForPrediction
import os
import torch
from transformers import PatchTSMixerForPrediction
from sklearn.metrics import mean_squared_error
import numpy as np
from torch.utils.data import DataLoader

for name in dfs:
    for pred_length in [1, 5, 10]:
        (X_train, y_train), (X_val, y_val), (X_test, y_test), scaler = get_data_for_period(dfs[name], window_size, pred_length)
        train_dataset = TimeSeriesDataset(X_train, y_train)
        valid_dataset = TimeSeriesDataset(X_val, y_val)
        test_dataset  = TimeSeriesDataset(X_test, y_test)

        results_folder = f"./20_pochs_patch_tsm_results_{name}_pred_length_{pred_length}"

        checkpoint_dirs = [os.path.join(results_folder, d) for d in os.listdir(results_folder) if d.startswith("checkpoint-")]

        best_checkpoint = None
        best_score = float("inf")
        
        val_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)


        for checkpoint_path in checkpoint_dirs:
            model = PatchTSMixerForPrediction.from_pretrained(checkpoint_path)
            model.eval()
            all_preds = []
            all_labels = []

            for batch in val_loader:

                with torch.no_grad():
                    outputs = model(**batch) 
                all_preds.append(outputs.prediction_outputs.flatten())
                all_labels.append(batch["future_values"].numpy().flatten())


            all_preds = np.concatenate(all_preds, axis=0)
            all_labels = np.concatenate(all_labels, axis=0)

            score = mean_squared_error(all_labels, all_preds)

            if score < best_score:
                best_score = score
                best_checkpoint = checkpoint_path

        print(f"\nBest model is from checkpoint: {best_checkpoint} with MSE: {best_score:.4f}")

        best_model = PatchTSMixerForPrediction.from_pretrained(best_checkpoint)


        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
        all_preds = []
        all_labels = []

        for batch in test_loader:
            with torch.no_grad():
                outputs = model(**batch)  
            all_preds.append(outputs.prediction_outputs.flatten())
            all_labels.append(batch["future_values"].numpy().flatten())


        all_preds = np.concatenate(all_preds, axis=0).reshape(-1, 1)
        all_labels = np.concatenate(all_labels, axis=0).reshape(-1, 1)
        
        print(all_preds.shape)

        preds_flat = scaler.inverse_transform(all_preds)
        labels_flat = scaler.inverse_transform(all_labels)

        mse  = mean_squared_error(labels_flat, preds_flat)
        rmse = np.sqrt(mse)
        mae  = mean_absolute_error(labels_flat, preds_flat)

        mask = labels_flat != 0
        mape = (np.mean(np.abs((labels_flat[mask] - preds_flat[mask]) / labels_flat[mask])) * 100
                if np.any(mask) else float('inf'))
        r2   = r2_score(labels_flat, preds_flat)

        
        print(f"{name} RMSE: {rmse} for {pred_length}")
        print(f"{name} MAE: {mae} for {pred_length}")
        print(f"{name} MAPE: {mape} for {pred_length}")
        print(f"{name} r2: {r2} for {pred_length}")
        


Best model is from checkpoint: ./20_pochs_patch_tsm_results_SPX_pred_length_1/checkpoint-4400 with MSE: 0.0008
(829, 1)
SPX RMSE: 94.84379577636719 for 1
SPX MAE: 75.76220703125 for 1
SPX MAPE: 1.6895703971385956 for 1
SPX r2: 0.9791280627250671 for 1

Best model is from checkpoint: ./20_pochs_patch_tsm_results_SPX_pred_length_5/checkpoint-4380 with MSE: 0.0013
(4125, 1)
SPX RMSE: 150.35609436035156 for 5
SPX MAE: 125.94324493408203 for 5
SPX MAPE: 2.8022438287734985 for 5
SPX r2: 0.9471405744552612 for 5

Best model is from checkpoint: ./20_pochs_patch_tsm_results_SPX_pred_length_10/checkpoint-4380 with MSE: 0.0019
(8200, 1)
SPX RMSE: 164.12283325195312 for 10
SPX MAE: 138.55908203125 for 10
SPX MAPE: 3.0737869441509247 for 10
SPX r2: 0.9365258812904358 for 10

Best model is from checkpoint: ./20_pochs_patch_tsm_results_AAPL_pred_length_1/checkpoint-4400 with MSE: 0.0051
(829, 1)
AAPL RMSE: 6.276037216186523 for 1
AAPL MAE: 5.052524089813232 for 1
AAPL MAPE: 2.898608334362507 for 1
A